# MLP idea

In [1]:
import os
import torch
import math
print(f"PyTorch has version {torch.__version__} with cuda {torch.version.cuda}")

PyTorch has version 2.3.1+cpu with cuda None


In [2]:
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Create a Recurrent Neural Network with LSTM

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [4]:
# Create a dataset
data = pd.read_csv(r"Data\Data4.csv")
t = data["0"].values
df = data.drop("0", axis=1)

# Input acceleration
freq1 = 1.75 # Freq1
t01 = 1.8 # Time Shift
wave = -2*freq1**2*np.pi**2*np.exp(-freq1**2*np.pi**2*(t01 - t)**2)*(t01 - t)*(2*freq1**2*np.pi**2*t01**2 - 4*freq1**2*np.pi**2*t01*t + 2*freq1**2*np.pi**2*t**2 - 3)
# Create acceleration array input
input_acc = np.zeros_like(df.values)
input_acc[:,0] = wave

# Node type
node_type = np.zeros(input_acc.shape[1])
node_type[[-1,0]] = 1

# Filter data
input_acc = input_acc[:,node_type==1]
df = df.iloc[:,node_type==1]

In [5]:
# Create a dataset
X = torch.tensor(input_acc, dtype=torch.float)
y = torch.tensor(df.values, dtype=torch.float)
print(f"X shape: {X.shape}, y shape: {y.shape}")

X shape: torch.Size([15000, 2]), y shape: torch.Size([15000, 2])


In [6]:
# Define train, validation and test data
train_size = int(0.8 * len(X))
val_size = int(0.1 * len(X))
test_size = len(X) - train_size - val_size

train_X = X[:train_size]
train_y = y[:train_size]

val_X = X[train_size:train_size+val_size]
val_y = y[train_size:train_size+val_size]

test_X = X[train_size+val_size:]
test_y = y[train_size+val_size:]

print(f"Train X shape: {train_X.shape}, Train y shape: {train_y.shape}")
print(f"Val X shape: {val_X.shape}, Val y shape: {val_y.shape}")
print(f"Test X shape: {test_X.shape}, Test y shape: {test_y.shape}")

Train X shape: torch.Size([12000, 2]), Train y shape: torch.Size([12000, 2])
Val X shape: torch.Size([1500, 2]), Val y shape: torch.Size([1500, 2])
Test X shape: torch.Size([1500, 2]), Test y shape: torch.Size([1500, 2])


In [7]:
# Create a dataset
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# Create a dataloader
train_dataset = TimeSeriesDataset(train_X, train_y)
val_dataset = TimeSeriesDataset(val_X, val_y)
test_dataset = TimeSeriesDataset(test_X, test_y)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [8]:
# Train the model
from tqdm import trange

def train(model, loss_function, optimizer, scheduler, train_dataloader, val_dataloader, epochs=100):
    model.train()
    train_losses = []
    val_losses = []
    
    for i in trange(epochs, desc="Training", unit="Epoch"):
        # Training phase
        model.train()
        for X, y in train_dataloader:
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                                 torch.zeros(1, 1, model.hidden_layer_size))

            y_pred = model(X)

            # Ensure y_pred and y have the same shape
            if y_pred.shape != y.shape:
                y_pred = y_pred.view_as(y)

            single_loss = loss_function(y_pred, y)
            single_loss.backward()
            optimizer.step()
        
        scheduler.step()
        train_losses.append(single_loss.item())

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_val, y_val in val_dataloader:
                y_val_pred = model(X_val)

                # Ensure y_val_pred and y_val have the same shape
                if y_val_pred.shape != y_val.shape:
                    y_val_pred = y_val_pred.view_as(y_val)

                val_loss += loss_function(y_val_pred, y_val).item()
        
        val_loss /= len(val_dataloader)
        val_losses.append(val_loss)

        if i % 10 == 0:
            print(f'Epoch: {i:3} Train Loss: {single_loss.item():10.8f} Val Loss: {val_loss:10.8f}')

    return train_losses, val_losses, model


In [9]:
# Define test function
def test(model, dataloader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for X, y in dataloader:
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                            torch.zeros(1, 1, model.hidden_layer_size))
            predictions.append(model(X).item())
    return predictions

In [10]:
model = LSTM(input_size=2, hidden_layer_size=100, output_size=2)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)


In [11]:
# Check size of input
for X, y in train_loader:
    print(f"X shape: {X.shape}, y shape: {y.shape}")
    pred = model(X)
    print(f"Prediction shape: {pred.shape}")
    print(f"Prediction: {pred}, y: {y}")
    loss = loss_function(pred, y.squeeze(0))
    print(f"Loss: {loss}")
    break

X shape: torch.Size([1, 2]), y shape: torch.Size([1, 2])
Prediction shape: torch.Size([2])
Prediction: tensor([-0.0498, -0.0351], grad_fn=<SelectBackward0>), y: tensor([[-4.0029e-38,  0.0000e+00]])
Loss: 0.0018576850416138768


In [12]:
# Assuming you have a validation dataloader named val_dataloader
train_losses, val_losses, model = train(model, loss_function, optimizer, scheduler, train_dataloader=train_loader, val_dataloader=val_loader, epochs=1000)

Training:   0%|          | 1/1000 [00:42<11:54:17, 42.90s/Epoch]

Epoch:   0 Train Loss: 0.05461305 Val Loss: 5.58844570


Training:   1%|          | 11/1000 [10:13<12:34:21, 45.76s/Epoch]

Epoch:  10 Train Loss: 0.00725264 Val Loss: 1.91171507


Training:   2%|▏         | 21/1000 [18:30<11:54:08, 43.77s/Epoch]

Epoch:  20 Train Loss: 0.15076622 Val Loss: 1.05674192


Training:   3%|▎         | 28/1000 [23:03<13:20:32, 49.42s/Epoch]


KeyboardInterrupt: 

In [13]:
# Plot the losses
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.legend()

NameError: name 'train_losses' is not defined